# Libraries

In [1]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from math import sqrt

# reading data
import os
import json
from collections import defaultdict

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import rfft, irfft, fftn, ifftn

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# eFormer
from eFormer.embeddings import SineActivation, CosineActivation
from eFormer.sparse_prob import SparseAttentionModule_Prob
from eFormer.sparse_det import SparseAttentionModule_Det

%store -r Kelmarsh_df Penmanshiel_df


# Architektur

## Hyperparameters

In [2]:
# set global parameters

n_heads_global = 4
probabilistic_model = True

## Embedding

probabilistic embedding & positional encoding

In [3]:
test_df = Kelmarsh_df['1'][['# Date and time', 'Energy Export (kWh)']][-1024:]

# First, ensure that the column is in datetime format
test_df['# Date and time'] = pd.to_datetime(test_df['# Date and time'])

# Then convert it to timestamps
test_df['Timestamp'] = test_df['# Date and time'].apply(lambda x: x.timestamp())

# interpolate NaN values
test_df = test_df.interpolate(method='linear')

features_matrix = test_df[['Energy Export (kWh)', 'Timestamp']].values

%store features_matrix

Stored 'features_matrix' (ndarray)


In [5]:
# dimensions of the tensor -> 2 for true values and time stamps
in_features = features_matrix.shape[-1]
# length of embedding vector
out_features = 64

encoding_model = Encoding(in_features, out_features)

# Forward pass through the model
feature_tensor = torch.tensor(features_matrix, dtype=torch.float32)

# check for NaN values early
if torch.isnan(feature_tensor).any():
    raise ValueError('NaN values detected in Input')

det_embeddings = encoding_model(feature_tensor)

# Check for NaN values after computation
if torch.isnan(det_embeddings).any():
    raise ValueError('NaN values detected in Embeddings')

print(det_embeddings.shape)

torch.Size([1, 1024, 64])


## Attention Mechanism

In the case of probabilistic modeling the embeddings are modified by alearnable random noise drawn from a gaussian distribution based on the embeddings itself. Therefor the two outputs have the same dimensions.

In [6]:
# determine which model to use
if probabilistic_model == True:
    model = SparseAttentionModule_Prob(
        d_model=embeddings.shape[-1],
        n_heads=n_heads_global,
        prob_sparse_factor=5
        )
else:
    model = SparseAttentionModule_Det(
        d_model=embeddings.shape[-1],
        n_heads=n_heads_global,
        prob_sparse_factor=5
    )

output_sparse = model(embeddings)

# check for NaN values early
if torch.isnan(output_sparse).any():
    raise ValueError('NaN values detected in ProbSparse Output')

In [ ]:
class SparseAttentionModule(nn.Module):
    def __init__(self, d_model, n_heads, prob_sparse_factor=5, attention_dropout=0.1):
        super(SparseAttentionModule, self).__init__()
        # Attention layers for both means and variances
        self.attention_layer_means = AttentionLayer(
            ProbAttention(mask_flag=True, factor=prob_sparse_factor, scale=None, attention_dropout=attention_dropout),
            d_model=d_model, n_heads=n_heads
        )
        self.attention_layer_vars = AttentionLayer(
            ProbAttention(mask_flag=True, factor=prob_sparse_factor, scale=None, attention_dropout=attention_dropout),
            d_model=d_model, n_heads=n_heads
        )

    def forward(self, embeddings):
        # Split embeddings into mean and variance
        means, variances = embeddings.split(embeddings.shape[-1] // 2, dim=-1)

        # Process means and variances separately through attention layers
        attention_output_means, _ = self.attention_layer_means(means, means, means, None)
        attention_output_vars, _ = self.attention_layer_vars(variances, variances, variances, None)

        # Combine the results
        combined_output = torch.cat([attention_output_means, attention_output_vars], dim=-1)

        return combined_output

# Example usage
model = SparseAttentionModule(
    d_model=(prob_embeddings.shape[-1] // 2),
    n_heads=3,
    prob_sparse_factor=5
    )

# Ensure prob_embeddings has the correct shape [B, L, E]
if len(prob_embeddings.shape) == 2:
    prob_embeddings = prob_embeddings.unsqueeze(0)  # Add batch dimension

elif len(prob_embeddings.shape) == 1:
    prob_embeddings = prob_embeddings.unsqueeze(0).unsqueeze(0)  # Add batch and length dimensions

output_mean_var = model(prob_embeddings)

# check for NaN values early
if torch.isnan(output).any():
    raise ValueError('NaN values detected in ProbSparse Output')

### Autocorrelation Attention

Credit to [Autoformer](https://github.com/thuml/Autoformer/blob/main/layers/AutoCorrelation.py)

In [7]:
class MovingAvg(nn.Module):
    """
    Moving average block to highlight the trend of time series.
    """
    def __init__(self, kernel_size, stride=1):
        super(MovingAvg, self).__init__()
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=(kernel_size - 1) // 2)

    def forward(self, x):
        x = self.avg(x.permute(0, 2, 1))
        return x.permute(0, 2, 1)

class SeriesDecomp(nn.Module):
    """
    Series decomposition block for extracting trend and seasonal components.
    """
    def __init__(self, kernel_size):
        super(SeriesDecomp, self).__init__()
        self.moving_avg = MovingAvg(kernel_size)

    def forward(self, x):
        trend = self.moving_avg(x)
        seasonal = x - trend
        return seasonal, trend

class SeasonalDecomp(nn.Module):
    """
    Special designed layernorm for the seasonal part
    """
    def __init__(self, channels):
        super(my_Layernorm, self).__init__()
        self.layernorm = nn.LayerNorm(channels)

    def forward(self, x):
        x_hat = self.layernorm(x)
        bias = torch.mean(x_hat, dim=1).unsqueeze(1).repeat(1, x.shape[1], 1)
        return x_hat - bias

class TriangularCausalMask():
    """ 
    Masking the future data points using a triangle.
    """ 
    def __init__(self, B, L, device="cpu"):
        mask_shape = [B, 1, L, L]
        with torch.no_grad():
            self._mask = torch.triu(torch.ones(mask_shape, dtype=torch.bool), diagonal=1).to(device)

    @property
    def mask(self):
        return self._mask

In [8]:
# initialize layer
series_decomp_layer = SeriesDecomp(kernel_size=7) 

# apply decomposition
seasonal, trend = series_decomp_layer(embeddings)

In [51]:
class AutoCorrelation(nn.Module):
    """
    AutoCorrelation Mechanism with the following two phases:
    (1) period-based dependencies discovery
    (2) time delay aggregation
    This block can replace the self-attention family mechanism seamlessly.
    """
    def __init__(self,d_model, mask_flag=False, factor=1, scale=None, attention_dropout=0.1, output_attention=False, top_k=2):
        super(AutoCorrelation, self).__init__()
        self.factor = factor
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)
        self.top_k = top_k
        self.d_model = d_model

    def time_delay_agg(self, values, corr):
        """
        Autocorrelation
        """
        batch = values.shape[0]
        head = values.shape[1]
        channel = self.top_k
        length = values.shape[3]
        # index init
        init_index = torch.arange(length).unsqueeze(0).unsqueeze(0).unsqueeze(0)\
            .repeat(batch, head, channel, 1).to(values.device)

        weights, delay = torch.topk(corr, self.top_k, dim=-1)
        # Reshape delay and weights dynamically
        delay = delay.reshape(batch, head, self.top_k, length)
        weights = weights.reshape(batch, head, self.top_k, length)
        # update corr
        tmp_corr = torch.softmax(weights, dim=-1)
        # print(f"tmp_corr non-zero:", len(tmp_corr[tmp_corr != 0]))
        # aggregation
        tmp_values = values.repeat(1, 1, 1, 2)
        delays_agg = torch.zeros_like(init_index).float()

        for i in range(self.top_k):
            tmp_delay = init_index.expand_as(delay) + delay[..., i].unsqueeze(-1)
            tmp_delay = torch.clamp(tmp_delay, min=0, max=length-1)
            pattern = torch.gather(tmp_values, dim=-1, index=tmp_delay)
            delays_agg = delays_agg + pattern * (tmp_corr[:,:,i,:].unsqueeze(1))
            
        # print(f"nonzero output: {len(delays_agg[delays_agg!=0])}")
        
        return delays_agg

    def forward(self, queries, keys, values, attn_mask):
        print("AR")
        print(f"queries shape: {queries.shape}")
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        if L > S:
            zeros = torch.zeros_like(queries[:, :(L - S), :]).float()
            values = torch.cat([values, zeros], dim=1)
            keys = torch.cat([keys, zeros], dim=1)
        else:
            values = values[:, :L, :, :]
            keys = keys[:, :L, :, :]

        # period-based dependencies
        q_fft = torch.fft.rfft(queries.permute(0, 2, 3, 1).contiguous(), dim=-1)
        k_fft = torch.fft.rfft(keys.permute(0, 2, 3, 1).contiguous(), dim=-1)
        res = q_fft * torch.conj(k_fft)
        corr = torch.fft.irfft(res, n=L, dim=-1)

        # Ensure corr has the shape [B, H, L, L]
        corr = corr.unsqueeze(2)
        expand_multiplier = L // corr.shape[3]
        corr = corr.expand(-1, -1, expand_multiplier, -1, -1)
        corr = corr.reshape(
            corr.shape[0],
            corr.shape[1],
            L,
            corr.shape[-1]
        )
        
        print(f"corr shape: {corr.shape}")
        print(f"values shape: {values.shape}")
        
        """
        # Create and apply the mask
        if self.mask_flag:
            mask = TriangularCausalMask(B, L, device=queries.device).mask
            mask = mask.expand(-1, H, -1, -1)  # Shape: [B, H, L, L]
            corr = corr.masked_fill(mask, 0)  # Zero out the masked positions
        """
        # time delay aggregation
        V = self.time_delay_agg(values.permute(0, 2, 3, 1).contiguous(), corr).permute(0, 3, 1, 2)

        # time delay agg
        if self.output_attention:
            return (V.contiguous(), corr.permute(0, 3, 1, 2))
        else:
            return (V.contiguous(), None)

class AutoCorrEncoderLayer(nn.Module):
    def __init__(self, attention, d_model, n_heads, d_ff=None, dropout=0.1, activation="relu"):
        super(AutoCorrEncoderLayer, self).__init__()
        self.attention = attention  # Use an instance of AutoCorrelation
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.d_v = d_model // n_heads

        # Projection layers for queries, keys, and values
        self.query_projection = nn.Linear(d_model, self.d_k * n_heads)
        self.key_projection = nn.Linear(d_model, self.d_k * n_heads)
        self.value_projection = nn.Linear(d_model, self.d_v * n_heads)

        # Output projection layer
        self.out_projection = nn.Linear(self.d_v * n_heads, d_model)

        # Optional feed forward layers
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff or 4 * d_model, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(in_channels=d_ff or 4 * d_model, out_channels=d_model, kernel_size=1, bias=False)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None):
        # Projection
        queries = self.query_projection(x).view(-1, x.size(1), self.n_heads, self.d_k).transpose(1,2)
        keys = self.key_projection(x).view(-1, x.size(1), self.n_heads, self.d_k).transpose(1,2)
        values = self.value_projection(x).view(-1, x.size(1), self.n_heads, self.d_v).transpose(1,2)

        # Apply AutoCorrelation Attention
        # Note: Adjust AutoCorrelation to handle multi-head input properly if needed
        attn_output, _ = self.attention(queries, keys, values, attn_mask)
        attn_output = attn_output.transpose(1, 2).contiguous().view(-1, x.size(1), self.d_model)
        x = x + self.dropout(self.out_projection(attn_output))
        x = self.norm1(x)

        # Optional Feed Forward
        y = self.dropout(self.activation(self.conv1(x.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))
        y = self.norm2(x + y)

        return y

class AutoCorrEncoder(nn.Module):
    def __init__(self, layer, N):
        super(AutoCorrEncoder, self).__init__()
        self.layers = nn.ModuleList([layer for _ in range(N)])
        self.norm = nn.LayerNorm(layer.attention.d_model)

    def forward(self, x, attn_mask=None):
        print("Encoder")
        for layer in self.layers:
            x = layer(x, attn_mask)
        return self.norm(x)

class MyAutoCorrelationModel(nn.Module):
    def __init__(self, d_model, n_heads, B, L, input_dim, top_k=4, device="cpu"):
        super(MyAutoCorrelationModel, self).__init__()
        self.device = device
        self.auto_corr = AutoCorrelation(
            mask_flag=False,
            factor=1,
            scale=None,
            attention_dropout=0.1,
            output_attention=False,
            top_k=top_k,
            d_model=d_model)
        self.encoder_layer = AutoCorrEncoderLayer(
            attention=self.auto_corr,
            d_model=d_model,
            n_heads=n_heads)
        self.encoder = AutoCorrEncoder(layer=self.encoder_layer, N=1)  # Adjust N for more layers

    def forward(self, input):
        print("Model")
        return self.encoder(input)

In [52]:
def AutoCorrelationFunction(input):
    B, L, _ = input.shape
    input_dim = input.shape[-1]

    model = MyAutoCorrelationModel(
        d_model=input_dim,
        n_heads=n_heads_global,
        B=B,
        L=L,
        input_dim=input_dim,
        top_k=4,  # Adjust as needed
        device=input.device
    )

    return model(input)

# Assume output_sparse is a properly formatted input tensor
output_ar_prob = AutoCorrelationFunction(embeddings)

# investigate number of dead nodes
print(len(output_ar_prob[output_ar_prob == 0]))
print(len(output_ar_prob[output_ar_prob != 0]))

Model
Encoder
AR
queries shape: torch.Size([1, 4, 1024, 16])


RuntimeError: shape '[1, 1024, 4, 4]' is invalid for input of size 0

## Loss Criterion

In [7]:
# CRPS (continouos ranked probability score)
def crps(forecast, observations, weights):
    """
    Args:
    forecast (pd.DataFrame or np.ndarray): Forecasts from the model (ensemble).
    observations (pd.Series or np.ndarray): Observed values.
    weights (np.array): Corresponding weights for the CRPS scores, derived from sparse attention.

    Returns:
    float: Weighted mean of the CRPS for all forecasts.
    """
    # Convert to NumPy arrays if input is Pandas
    if isinstance(forecast, pd.DataFrame):
        forecast = forecast.to_numpy()
    if isinstance(observations, pd.Series):
        observations = observations.to_numpy()
    
    # Sort forecast samples
    forecast.sort(axis=0)

    # Ensure observations are broadcastable over the forecast_samples
    observations = observations[np.newaxis, :]

    # Calculate CRPS
    cumsum_forecast = np.cumsum(forecast, axis=0) / forecast.shape[0]
    crps = np.mean((cumsum_forecast - (forecast > observations).astype(float)) ** 2, axis=0)
    
    # weighted median of CRPS
    if len(crps) != len(weights):
        raise ValueError("Length of CRPS series and weights must be equal")

    weighted_sum = np.sum(crps * weights)
    total_weights = np.sum(weights)

    if total_weights == 0:
        raise ValueError("Total weight cannot be zero")

    weighted_crps = weighted_sum / total_weights
    
    return round(weighted_crps, 4)

# Example usage
forecast_samples = pd.DataFrame(np.random.randn(1000, 5))  # Example forecast samples
observations = pd.Series(np.random.randn(5))  # Example observations
weights = np.random.rand(5)  # Example weights

weighted_crps = crps(forecast_samples, observations, weights)
print("Weighted Mean CRPS:", weighted_crps)

Weighted Mean CRPS: 1.0589


# Transformer Model

In [8]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, sine_activation, cosine_activation, model):
        super(TimeSeriesTransformer, self).__init__()
        self.sine_activation = sine_activation
        self.cosine_activation = cosine_activation
        self.sparse_attention = model

    def forward(self, input_series, future_time_stamps):
        # Assuming future_time_stamps is preprocessed to match the input dimensionality
        # and choosing either sine or cosine activation based on the use case
        prediction_encodings = self.Encoding(prediction_time_stamps)  # or cosine_activation

        # Use the Sparse Attention Module for predicting future values
        future_predictions = self.sparse_attention(prediction_encodings)

        # future_predictions could be further processed to predict specific values, e.g., through a linear layer
        linear_layer = nn.Linear(in_features, out_features)(future_predictions)
        predictions = softmax(linear_layer)
        
        return future_predictions

In [11]:
test

TimeSeriesTransformer(
  (sine_activation): SineActivation()
  (cosine_activation): CosineActivation()
  (sparse_attention): SparseAttentionModule_Prob(
    (sampler): ProbabilisticEmbeddingSampler()
    (attention_layer): AttentionLayer(
      (inner_attention): ProbAttention(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (query_projection): Linear(in_features=64, out_features=64, bias=True)
      (key_projection): Linear(in_features=64, out_features=64, bias=True)
      (value_projection): Linear(in_features=64, out_features=64, bias=True)
      (out_projection): Linear(in_features=64, out_features=64, bias=True)
    )
  )
)

# Test Area

### AutoCorr Layers

In [ ]:
class seasonal_layer(nn.Module):
    """
    Special designed layernorm for the seasonal part
    """
    def __init__(self, channels):
        super(my_Layernorm, self).__init__()
        self.layernorm = nn.LayerNorm(channels)

    def forward(self, x):
        x_hat = self.layernorm(x)
        bias = torch.mean(x_hat, dim=1).unsqueeze(1).repeat(1, x.shape[1], 1)
        return x_hat - bias


class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean


class EncoderLayer(nn.Module):
    """
    Autoformer encoder layer with the progressive decomposition architecture
    """
    def __init__(self, attention, d_model, d_ff=None, moving_avg=25, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1, bias=False)
        self.decomp1 = series_decomp(moving_avg)
        self.decomp2 = series_decomp(moving_avg)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None):
        new_x, attn = self.attention(
            x, x, x,
            attn_mask=attn_mask
        )
        x = x + self.dropout(new_x)
        x, _ = self.decomp1(x)
        y = x
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))
        res, _ = self.decomp2(x + y)
        return res, attn


class Encoder(nn.Module):
    """
    Autoformer encoder
    """
    def __init__(self, attn_layers, conv_layers=None, norm_layer=None):
        super(Encoder, self).__init__()
        self.attn_layers = nn.ModuleList(attn_layers)
        self.conv_layers = nn.ModuleList(conv_layers) if conv_layers is not None else None
        self.norm = norm_layer

    def forward(self, x, attn_mask=None):
        attns = []
        if self.conv_layers is not None:
            for attn_layer, conv_layer in zip(self.attn_layers, self.conv_layers):
                x, attn = attn_layer(x, attn_mask=attn_mask)
                x = conv_layer(x)
                attns.append(attn)
            x, attn = self.attn_layers[-1](x)
            attns.append(attn)
        else:
            for attn_layer in self.attn_layers:
                x, attn = attn_layer(x, attn_mask=attn_mask)
                attns.append(attn)

        if self.norm is not None:
            x = self.norm(x)

        return x, attns

In [ ]:
# probabilistic masking
class ProbMask(nn.Module):
    def __init__(self, B, H, L, input_dim, top_k=5, device="cpu"):
        super(ProbMask, self).__init__()
        self.B = B
        self.H = H
        self.L = L
        self.top_k = top_k
        self.device = device
        self.scoring_network = ScoringNetwork(input_dim, L).to(device)

    def forward(self, input_sequence):
        # Compute scores using the scoring network
        scores = self.scoring_network(input_sequence)  # input_sequence shape: [B, L, input_dim]

        # Select top-k scores to generate indices
        _, indices = torch.topk(scores, self.top_k, dim=-1)  # Selecting indices of top-k scores

        # Create a mask for all positions
        full_mask = torch.ones((self.B, self.L), dtype=torch.bool).to(self.device)

        # Update mask for top-k positions
        batch_indices = torch.arange(self.B, device=self.device)[:, None]
        full_mask[batch_indices, indices] = False

        # Expand mask for all heads
        mask = full_mask[:, None, :].expand(-1, self.H, -1)

        # Mask should be of shape [B, H, L, L]
        mask = mask.unsqueeze(2).expand(-1, -1, self.L, -1)

        return mask

class ScoringNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ScoringNetwork, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # Using sigmoid to keep scores between 0 and 1